# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

## Dependencies

First we will be importing all the needed dependencies to complete the project.

Warning: Install the `imblearn` library before running the rest of the notebook

In [1]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

from azureml.core.dataset import Dataset
from azureml.core import Datastore
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.data.dataset_factory import TabularDatasetFactory

from azureml.widgets import RunDetails

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.27.0


## Workspace Configuration

In this cell we import the workspace configuration and create an experiment that we will use later.

In [2]:
ws = Workspace.from_config()

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

# choose a name for experiment
experiment_name = 'hyperdrivecovid'

experiment=Experiment(ws, experiment_name)
experiment

Workspace name: quick-starts-ws-144154
Azure region: southcentralus
Subscription id: 976ee174-3882-4721-b90a-b5fef6b72f24
Resource group: aml-quickstarts-144154


Name,Workspace,Report Page,Docs Page
hyperdrivecovid,quick-starts-ws-144154,Link to Azure Machine Learning studio,Link to Documentation


## Compute Cluster creation
In this cell a cpu cluster is created for running our experiments, it checks if a compute cluster with the same name exists, if it exists then uses it, if not it creates it.

If the cluster does not exists we define the configuration for it. For this project we will be using `min_nodes = 1`, in your own project this will incurr in extra costs, so consider leaving it to 0.

In [3]:
compute_cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print("Found existing compute cluster...")
except:
    print("Creating new compute cluster...")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4, min_nodes=1)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)
    
compute_target.wait_for_completion(show_output=True)
print("Cluster details: ", compute_target.get_status().serialize())

Found existing compute cluster...
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
Cluster details:  {'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 1, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-05-07T15:31:12.423000+00:00', 'errors': None, 'creationTime': '2021-05-07T15:29:18.820325+00:00', 'modifiedTime': '2021-05-07T15:29:34.169299+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Dataset

### Overview
We are going to use the Mexican Government's COVID-19 data, once it is uploaded to ML Studio we will consume it using TabularDataset.

To consume the dataset we will import it from the Datastore tab in ML Studio, this is a blob storage, then we have to specify the location of the csv to import it.

Once it's imported we will we creating a Pandas Dataframe that we will need later on.

In [4]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

from azureml.core.dataset import Dataset
from azureml.core import Datastore
from azureml.data.dataset_factory import TabularDatasetFactory

ws = Workspace.from_config()
datastore = Datastore.get(ws,'workspaceblobstore')
ds = TabularDatasetFactory.from_delimited_files(path=(datastore, 'UI/05-07-2021_031820_UTC/210506COVID19MEXICO.csv'))
df = ds.to_pandas_dataframe()

print(df)

        FECHA_ACTUALIZACION ID_REGISTRO  ORIGEN  SECTOR  ENTIDAD_UM  SEXO  \
0                2021-05-06      z482b8       1      12           9     2   
1                2021-05-06      z49a69       1      12          23     1   
2                2021-05-06      z23d9d       1      12          22     2   
3                2021-05-06      z24953       1      12           9     1   
4                2021-05-06      zz8e77       1      12           9     2   
...                     ...         ...     ...     ...         ...   ...   
6714847          2021-05-06      9f2664       2       4           9     1   
6714848          2021-05-06      79560e       2      12          17     1   
6714849          2021-05-06      bb7733       2       4          15     1   
6714850          2021-05-06      659996       2      12          21     1   
6714851          2021-05-06      d0ecac       2       4           8     2   

         ENTIDAD_NAC  ENTIDAD_RES  MUNICIPIO_RES  TIPO_PACIENTE  ...  \
0  

### Data Preprocessing

#### Cleaning
The objective of our project is to create a model that can get if a patient with COVID-19 will enter into an Intensive Care Unit or not.

The objective variable `UCI` has a lot of values that are 97, 98, or 99, which means that we don't know if the person is or was in a Intensive Care Unit, we only need the values 1 and 2, which means that the patient was or wasn't in an Intensive Care Unit, respectively.

So the next step is to trim our dataset by removing the rows that have 97, 98 or 99, leaving only 'Yes'/'No' for our objective variable.

In [5]:
arr_drop = [97,98,99]
for element in arr_drop:
    df.drop(df.loc[df['UCI']==element].index,inplace=True)

print(df)
print("--------------------------------------")
print("Classes count")
classes = pd.crosstab(index=df['UCI'],columns="count")
print(classes)

        FECHA_ACTUALIZACION ID_REGISTRO  ORIGEN  SECTOR  ENTIDAD_UM  SEXO  \
1                2021-05-06      z49a69       1      12          23     1   
16               2021-05-06      z166d5       1      12           1     1   
21               2021-05-06      z388cd       1      12          29     2   
22               2021-05-06      z4533d       1      12          29     1   
27               2021-05-06      z4494e       1      12           8     1   
...                     ...         ...     ...     ...         ...   ...   
6714797          2021-05-06      9049b2       2       4           9     1   
6714818          2021-05-06      974d9b       1      12           9     1   
6714825          2021-05-06      e48121       2       4          26     2   
6714832          2021-05-06      a74bfa       1       4           9     2   
6714842          2021-05-06      e20865       1       4           5     1   

         ENTIDAD_NAC  ENTIDAD_RES  MUNICIPIO_RES  TIPO_PACIENTE  ...  \
1  

#### Handling dataset class imbalance

As we can see, the rows with 1 (Yes) in the `UCI` column are a lot less than the 2 (No). we can send the dataset to training like this, but we will get a "Class balancing warning" alert in Azure ML, so we are going to fix it.

The next step in our data preprocessing is to balance the dataset. We will need the `imblearn` library that we installed before, and we will need to divide our dataframe in X and y to use the "RandomUnderSampler" method.

`X` has all the columns except `UCI` and `y` has only `UCI`


In [6]:
y = df['UCI']
X = df.drop('UCI',1)

rus = RandomUnderSampler(random_state = 42)
X_resampled, y_resampled = rus.fit_resample(X,y)

### Train/test split

Now that we have our balanced data, we will divide X and y between train and test

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X_resampled,y_resampled,test_size = 0.2, random_state = 42)
train_data = pd.concat([x_train,y_train],axis=1)

print(train_data)

print("Validating that we have balanced data for training.")
classes = pd.crosstab(index=train_data['UCI'],columns="count")
print(classes)

       FECHA_ACTUALIZACION ID_REGISTRO  ORIGEN  SECTOR  ENTIDAD_UM  SEXO  \
40588           2021-05-06      2d79b8       2      12          13     2   
34614           2021-05-06      318c56       2      12           8     2   
60846           2021-05-06      0f5dbb       1      12           8     1   
103582          2021-05-06      1c4c4b       2       4          15     2   
7096            2021-05-06      1ba463       2       9          25     1   
...                    ...         ...     ...     ...         ...   ...   
54886           2021-05-06      05613c       1      12          31     2   
76820           2021-05-06      146527       1      12          22     2   
103694          2021-05-06      07b674       1       4           9     1   
860             2021-05-06      0f79f4       1       4          31     1   
15795           2021-05-06      1d5e74       2       9          25     2   

        ENTIDAD_NAC  ENTIDAD_RES  MUNICIPIO_RES  TIPO_PACIENTE  ... OTRO_CASO  \
40588 

### Converting Train Data to Tabular Dataset
Since the train data is in the pandas Dataframe format, we have to convert it to Tabular Dataset to be used in AutoML

In [8]:
#Convert train_data (Which are in pandas DataFrame format) to TabularDataset format.
try:
    os.makedirs('./data', exist_ok=True)
except OSError as error:
    print('New directory cannot be created')
    
path = 'data/trainBalanced.csv'
train_data.to_csv(path)

datastore = ws.get_default_datastore()
datastore.upload(src_dir='data', target_path='data')

train_data = TabularDatasetFactory.from_delimited_files(path=[(datastore, ('data/trainBalanced.csv'))])
print("Successfully converted the dataset to TabularDataset format.")
print(type(train_data))

Uploading an estimated of 1 files
Uploading data/trainBalanced.csv
Uploaded data/trainBalanced.csv, 1 files out of an estimated total of 1
Uploaded 1 files
Successfully converted the dataset to TabularDataset format.
<class 'azureml.data.tabular_dataset.TabularDataset'>


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = <your policy here>

#TODO: Create the different params that you will be using during training
param_sampling = <your params here>

#TODO: Create your estimator and hyperdrive config
estimator = <your estimator here>

hyperdrive_run_config = <your config here?

In [ ]:
#TODO: Submit your experiment

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service